In [3]:
import numpy as np
import networkx as nx
import copy
import pickle
import sys
import time
sys.path.append("/home/ubuntu/CE_scheme/")
from BayesCard.Evaluation.utils import parse_query
from Schemas.stats.schema import gen_stats_light_schema
from Join_scheme.data_prepare import identify_key_values, process_stats_data

In [9]:
data_path = "/home/ubuntu/End-to-End-CardEst-Benchmark/datasets/stats_simplified/{}.csv"
schema = gen_stats_light_schema(data_path)
all_keys, equivalent_keys = identify_key_values(schema)
print(equivalent_keys)
model_path = "/home/ubuntu/data_CE/CE_scheme_models/model_stats_greedy_200.pkl"
with open(model_path, "rb") as f:
    new_BE = pickle.load(f)

{'posts.Id': {'tags.ExcerptPostId', 'comments.PostId', 'postHistory.PostId', 'posts.Id', 'postLinks.RelatedPostId', 'votes.PostId', 'postLinks.PostId'}, 'users.Id': {'comments.UserId', 'posts.OwnerUserId', 'badges.UserId', 'postHistory.UserId', 'users.Id', 'votes.UserId'}}


In [3]:
import numpy as np
import copy

from Join_scheme.join_graph import process_condition, get_join_hyper_graph
from Join_scheme.data_prepare import identify_key_values
from BayesCard.Evaluation.cardinality_estimation import timestamp_transorform, construct_table_query


class Factor:
    """
    This the class defines a multidimensional conditional probability.
    """
    def __init__(self, variables, pdfs, equivalent_variables=[]):
        self.variables = variables
        self.equivalent_variables = equivalent_variables
        self.pdfs = pdfs
        self.cardinalities = dict()
        for i, var in enumerate(self.variables):
            self.cardinalities[var] = pdfs.shape[i]
            if len(equivalent_variables) != 0:
                self.cardinalities[equivalent_variables[i]] = pdfs.shape[i]


class Bound_ensemble:
    """
    This the class where we store all the trained models and perform inference on the bound.
    """
    def __init__(self, bns, table_buckets, schema):
        self.bns = bns
        self.table_buckets = table_buckets
        self.schema = schema
        self.all_keys, self.equivalent_keys = identify_key_values(schema)

    def parse_query_simple(self, query):
        """
        If your selection query contains no aggregation and nested sub-queries, you can use this function to parse a
        join query. Otherwise, use parse_query function.
        """
        query = query.replace(" where ", " WHERE ")
        query = query.replace(" from ", " FROM ")
        query = query.replace(" and ", " AND ")
        query = query.split(";")[0]
        query = query.strip()
        tables_all = {}
        join_cond = []
        table_query = {}
        join_keys = {}
        tables_str = query.split(" WHERE ")[0].split(" FROM ")[-1]
        for table_str in tables_str.split(","):
            table_str = table_str.strip()
            if " as " in table_str:
                tables_all[table_str.split(" as ")[-1]] = table_str.split(" as ")[0]
            else:
                tables_all[table_str.split(" ")[-1]] = table_str.split(" ")[0]

        # processing conditions
        conditions = query.split(" WHERE ")[-1].split(" AND ")
        for cond in conditions:
            table, cond, join, join_key = process_condition(cond, tables_all)
            if not join:
                attr = cond[0]
                op = cond[1]
                value = cond[2]
                if "Date" in attr:
                    assert "::timestamp" in value
                    value = timestamp_transorform(value.strip().split("::timestamp")[0])
                if table not in table_query:
                    table_query[table] = dict()
                construct_table_query(self.bns[table], table_query[table], attr, op, value)
            else:
                join_cond.append(cond)
                for tab in join_key:
                    if tab in join_keys:
                        join_keys[tab].add(join_key[tab])
                    else:
                        join_keys[tab] = set([join_key[tab]])

        return tables_all, table_query, join_cond, join_keys

    def get_all_id_conidtional_distribution(self, table_queries, join_keys, equivalent_group):
        res = dict()
        for table in join_keys:
            key_attrs = list(join_keys[table])
            if table in table_queries:
                table_query = table_queries[table]
            else:
                table_query = {}
            id_attrs, probs = self.bns[table].query_id_prob(table_query, key_attrs)
            new_id_attrs = []
            for K in id_attrs:
                for PK in equivalent_group:
                    if K in equivalent_group[PK]:
                        new_id_attrs.append(PK)
            assert len(new_id_attrs) == len(id_attrs)
            res[table] = Factor(id_attrs, probs, new_id_attrs)
        return res

    def eliminate_one_key_group_general(self, tables, key_group, factors):
        rest_groups = dict()
        rest_group_tables = dict()
        for table in tables:
            assert key_group in factors[table].equivalent_variables
            temp = copy.deepcopy(factors[table].equivalent_variables)
            temp.remove(key_group)
            for keys in temp:
                if keys in rest_groups:
                    assert factors[table].cardinalities[keys] == rest_groups[keys]
                    rest_group_tables[keys].append(table)
                else:
                    rest_groups[keys] = factors[table].cardinalities[keys]
                    rest_group_tables[keys] = [table]

        new_factor_variables = []
        new_factor_cardinalities = []
        for key in rest_groups:
            new_factor_variables.append(key)
            new_factor_cardinalities.append(rest_groups[key])
        new_factor_pdf = np.zeros(tuple(new_factor_cardinalities))

    def eliminate_one_key_group(self, tables, key_group, factors, relevant_keys):
        """This version only supports 2D distributions"""
        rest_group = None
        rest_group_cardinalty = 0
        eliminated_tables = []
        rest_group_tables = []
        for table in tables:
            assert key_group in factors[table].equivalent_variables
            temp = copy.deepcopy(factors[table].equivalent_variables)
            temp.remove(key_group)
            if len(temp) == 0:
                eliminated_tables.append(table)
            for key in temp:
                if rest_group:
                    assert factors[table].cardinalities[key] == rest_group_cardinalty
                    rest_group_tables.append(table)
                else:
                    rest_group = key
                    rest_group_cardinalty = factors[table].cardinalities[key]
                    rest_group_tables = [table]

        all_probs_eliminated = []
        all_modes_eliminated = []
        for table in eliminated_tables:
            bin_modes = self.table_buckets[table].oned_bin_modes[relevant_keys[key_group][table]]
            all_probs_eliminated.append(factors[table].pdfs)
            all_modes_eliminated.append(np.minimum(bin_modes, factors[table].pdfs))

        if rest_group:
            new_factor_pdf = np.zeros(rest_group_cardinalty)
        else:
            return self.compute_bound_oned(all_probs_eliminated, all_modes_eliminated)

        for i in range(rest_group_cardinalty):
            for table in rest_group_tables:
                idx_f = factors[table].equivalent_variables.index(key_group)
                idx_b = self.table_buckets[table].id_attributes.index(relevant_keys[key_group][table])
                bin_modes = self.table_buckets[table].twod_bin_modes[relevant_keys[key_group][table]]
                if idx_f == 0 and idx_b == 0:
                    all_probs_eliminated.append(factors[table].pdfs[:, i])
                    all_modes_eliminated.append(np.minimum(bin_modes[:, i], factors[table].pdfs[:, i]))
                elif idx_f == 0 and idx_b == 1:
                    all_probs_eliminated.append(factors[table].pdfs[:, i])
                    all_modes_eliminated.append(np.minimum(bin_modes[i, :], factors[table].pdfs[:, i]))
                elif idx_f == 1 and idx_b == 0:
                    all_probs_eliminated.append(factors[table].pdfs[i, :])
                    all_modes_eliminated.append(np.minimum(bin_modes[:, i], factors[table].pdfs[i, :]))
                else:
                    all_probs_eliminated.append(factors[table].pdfs[i, :])
                    all_modes_eliminated.append(np.minimum(bin_modes[i, :], factors[table].pdfs[i, :]))
            new_factor_pdf[i] = self.compute_bound_oned(all_probs_eliminated, all_modes_eliminated)

        for table in rest_group_tables:
            factors[table] = Factor([rest_group], new_factor_pdf, [rest_group])

        return None

    def compute_bound_oned(self, all_probs, all_modes):
        all_probs = np.stack(all_probs, axis=0)
        all_modes = np.stack(all_modes, axis=0)
        multiplier = np.prod(all_modes, axis=0)
        non_zero_idx = np.where(multiplier != 0)[0]
        min_number = np.amin(all_probs[:, non_zero_idx]/all_modes[:, non_zero_idx], axis=0)
        multiplier[non_zero_idx] = multiplier[non_zero_idx] * min_number
        return np.sum(multiplier)

    def get_optimal_elimination_order(self, equivalent_group, join_keys, factors):
        cardinalities = dict()
        lengths = dict()
        tables_involved = dict()
        relevant_keys = dict()
        for group in equivalent_group:
            relevant_keys[group] = dict()
            lengths[group] = len(equivalent_group[group])
            cardinalities[group] = []
            tables_involved[group] = set([])
            for keys in equivalent_group[group]:
                for table in join_keys:
                    if keys in join_keys[table]:
                        cardinalities[group].append(len(join_keys[table]))
                        tables_involved[group].add(table)
                        variables = factors[table].variables
                        variables[variables.index(keys)] = group
                        factors[table].variables = variables
                        relevant_keys[group][table] = keys
                        break
            cardinalities[group] = np.asarray(cardinalities[group])

        optimal_order = list(equivalent_group.keys())
        for i in range(len(optimal_order)):
            min_idx = i
            for j in range(i+1, len(optimal_order)):
                min_group = optimal_order[min_idx]
                curr_group = optimal_order[j]
                if np.max(cardinalities[curr_group]) < np.max(cardinalities[min_group]):
                    min_idx = j
                else:
                    min_max_tables = np.max(cardinalities[min_group])
                    min_num_max_tables = len(np.where(cardinalities[min_group] == min_max_tables)[0])
                    curr_max_tables = np.max(cardinalities[curr_group])
                    curr_num_max_tables = len(np.where(cardinalities[curr_group] == curr_max_tables)[0])
                    if curr_num_max_tables < min_num_max_tables:
                        min_idx = j
                    elif lengths[curr_group] < lengths[min_group]:
                        min_idx = j
            optimal_order[i], optimal_order[min_idx] = optimal_order[min_idx], optimal_order[i]
        return optimal_order, tables_involved, relevant_keys

    def get_cardinality_bound(self, query_str):
        tables_all, table_queries, join_cond, join_keys = self.parse_query_simple(query_str)
        equivalent_group = get_join_hyper_graph(join_keys, self.equivalent_keys)
        conditional_factors = self.get_all_id_conidtional_distribution(table_queries, join_keys, equivalent_group)
        optimal_order, tables_involved, relevant_keys = self.get_optimal_elimination_order(equivalent_group, join_keys,
                                                                            conditional_factors)

        for key_group in optimal_order:
            tables = tables_involved[key_group]
            res = self.eliminate_one_key_group(tables, key_group, conditional_factors, relevant_keys)
        return res

In [10]:
query_str = "SELECT COUNT(*) FROM users as u, comments as c, posts as p WHERE p.OwnerUserId = u.Id AND p.Id = c.PostId AND u.UpVotes>=0 AND u.CreationDate>='2010-08-21 21:27:38'::timestamp AND c.CreationDate>='2010-07-21 11:05:37'::timestamp AND c.CreationDate<='2014-08-25 17:59:25'::timestamp"
t = time.time()
res = new_BE.get_cardinality_bound(query57)
print(time.time() - t)
print(res)

0.1431887149810791
0.0


In [12]:
query_file = "/home/ubuntu/End-to-End-CardEst-Benchmark/workloads/stats_CEB/sub_plan_queries/stats_CEB_sub_queries.sql"
with open(query_file, "r") as f:
    queries = f.readlines()


In [13]:
qerror = []
latency = []
pred = []
for i, query_str in enumerate(queries):
    query = query_str.split("||")[0][:-1]
    print("========================")
    true_card = int(query_str.split("||")[-1])
    t = time.time()
    res = new_BE.get_cardinality_bound(query)
    pred.append(res)
    latency.append(time.time() - t)
    qerror.append(res/true_card)
    print(f"estimating query {i}: predicted {res}, true_card {true_card}, qerror {res/true_card}, latency {time.time() - t}")

estimating query 0: predicted 79851.0, true_card 79851, qerror 1.0, latency 0.014549016952514648
estimating query 1: predicted 12295160.0, true_card 10220614, qerror 1.202976650913536, latency 0.022883176803588867
estimating query 2: predicted 4126975.0, true_card 1458075, qerror 2.830427104229892, latency 0.013505697250366211
estimating query 3: predicted 4242223.095235909, true_card 1709781, qerror 2.4811499807495285, latency 0.01455831527709961
estimating query 4: predicted 10719765.0, true_card 7491903, qerror 1.4308467421428175, latency 0.013917684555053711
estimating query 5: predicted 1540892.1294773642, true_card 428612, qerror 3.5950746350483986, latency 0.01465296745300293
estimating query 6: predicted 61288345.969291344, true_card 55900138, qerror 1.0963898867171193, latency 0.013592243194580078
estimating query 7: predicted 11101.71481716133, true_card 10972, qerror 1.011822349358488, latency 0.009617090225219727
estimating query 8: predicted 0.0, true_card 699302, qerror 0

estimating query 57: predicted 315172.02462900564, true_card 288438, qerror 1.0926855151852586, latency 0.011746644973754883
estimating query 58: predicted 34488.66194645628, true_card 31965, qerror 1.0789507882514089, latency 0.012324094772338867
estimating query 59: predicted 0.0, true_card 26836, qerror 0.0, latency 0.06852054595947266
estimating query 60: predicted 3777094.372477355, true_card 2704241, qerror 1.3967299410360818, latency 0.006231784820556641
estimating query 61: predicted 34523.77321486599, true_card 32918, qerror 1.0487810078032078, latency 0.005340099334716797
estimating query 62: predicted 88785.02229608594, true_card 86112, qerror 1.0310412288192812, latency 0.005836963653564453
estimating query 63: predicted 3542182.6967073386, true_card 2488080, qerror 1.42366109478286, latency 0.008292436599731445
estimating query 64: predicted 1605160.0, true_card 1056687, qerror 1.5190496334297667, latency 0.004283428192138672
estimating query 65: predicted 29153.0, true_ca

estimating query 114: predicted 0.0, true_card 2112679, qerror 0.0, latency 0.06756424903869629
estimating query 115: predicted 0.8955163459981166, true_card 59978, qerror 1.4930747040550145e-05, latency 0.06310057640075684
estimating query 116: predicted 244298.0312973327, true_card 80977, qerror 3.0168817231724154, latency 0.01307988166809082
estimating query 117: predicted 0.0, true_card 595820, qerror 0.0, latency 0.07090234756469727
estimating query 118: predicted 2521891.827024441, true_card 1187068, qerror 2.124471240926755, latency 0.008429527282714844
estimating query 119: predicted 1605160.0, true_card 1056687, qerror 1.5190496334297667, latency 0.0042378902435302734
estimating query 120: predicted 34773.0, true_card 34789, qerror 0.9995400845094714, latency 0.003160715103149414
estimating query 121: predicted 2187293.5976249976, true_card 1486002, qerror 1.471931799301076, latency 0.007307529449462891
estimating query 122: predicted 36745.07021592576, true_card 37048, qerror

estimating query 177: predicted 1019.632783676863, true_card 913, qerror 1.1167938484960165, latency 0.012439489364624023
estimating query 178: predicted 76886.70027040935, true_card 68895, qerror 1.1159982621439777, latency 0.005329132080078125
estimating query 179: predicted 106875434.3350742, true_card 91539518, qerror 1.167533286935968, latency 0.0128936767578125
estimating query 180: predicted 220767.87643332206, true_card 70700, qerror 3.1226007982082327, latency 0.01508951187133789
estimating query 181: predicted 4922955.061389457, true_card 1984289, qerror 2.480966765118114, latency 0.008415937423706055
estimating query 182: predicted 46228.29531424531, true_card 18346, qerror 2.519802426373341, latency 0.012274503707885742
estimating query 183: predicted 19197329.547721237, true_card 3169724, qerror 6.056467234283249, latency 0.015708446502685547
estimating query 184: predicted 481485.2622397495, true_card 345663, qerror 1.3929326026787636, latency 0.012134790420532227
estimat

estimating query 235: predicted 0.0, true_card 28565, qerror 0.0, latency 0.06457829475402832
estimating query 236: predicted 4541.122358181983, true_card 560, qerror 8.109147068182113, latency 0.007525444030761719
estimating query 237: predicted 1255.0431330975925, true_card 1091, qerror 1.1503603419776283, latency 0.011205434799194336
estimating query 238: predicted 72636.2061261739, true_card 22791, qerror 3.187056562949142, latency 0.009663581848144531
estimating query 239: predicted 63224.18367545947, true_card 53752, qerror 1.1762201160042318, latency 0.007602214813232422
estimating query 240: predicted 1.00877866711461, true_card 332, qerror 0.003038489961188584, latency 0.05822587013244629
estimating query 241: predicted 0.0, true_card 11332, qerror 0.0, latency 0.0632171630859375
estimating query 242: predicted 24612.95040407141, true_card 5044, qerror 4.879649168134696, latency 0.013038158416748047
estimating query 243: predicted 0.0, true_card 1717, qerror 0.0, latency 0.068

...

estimating query 2316: predicted 105832.20736942571, true_card 2821168, qerror 0.037513613995843464, latency 0.059664249420166016
estimating query 2317: predicted 18448884.380521927, true_card 348955301, qerror 0.05286890420536104, latency 0.06017613410949707
estimating query 2318: predicted 16918841.08988168, true_card 3384567, qerror 4.998819964232258, latency 0.05913424491882324
estimating query 2319: predicted 0.0, true_card 69119371, qerror 0.0, latency 0.0646064281463623
estimating query 2320: predicted 0.0, true_card 71466602, qerror 0.0, latency 0.06464958190917969
estimating query 2321: predicted 6700500.873217395, true_card 1000743441, qerror 0.006695523146793619, latency 0.057276010513305664
estimating query 2322: predicted 6700500.873217395, true_card 8872062, qerror 0.7552360289206043, latency 0.056085824966430664
estimating query 2323: predicted 911040.8542911514, true_card 23808461, qerror 0.03826542397222363, latency 0.05679941177368164
estimating query 2324: predicted 

estimating query 2372: predicted 2036852.2203597452, true_card 21200677, qerror 0.09607486687145629, latency 0.0894162654876709
estimating query 2373: predicted 2036852.2203597452, true_card 246477, qerror 8.26386324224875, latency 0.08655691146850586
estimating query 2374: predicted 0.0, true_card 5543342, qerror 0.0, latency 0.06167411804199219
estimating query 2375: predicted 59682020.32281356, true_card 4030299, qerror 14.808335640311938, latency 0.043752431869506836
estimating query 2376: predicted 972834.2761617153, true_card 12304064, qerror 0.07906609362253929, latency 0.06029009819030762
estimating query 2377: predicted 972834.2761617153, true_card 133378, qerror 7.2938136436422445, latency 0.05820107460021973
estimating query 2378: predicted 820554.1041146084, true_card 44869387, qerror 0.018287615654624573, latency 0.08428478240966797
estimating query 2379: predicted 820554.1041146084, true_card 504508, qerror 1.6264441874353002, latency 0.08462285995483398
estimating query 

estimating query 2429: predicted 105832.20736942571, true_card 28885, qerror 3.6639157822200352, latency 0.07556772232055664
estimating query 2430: predicted 6974623.48066115, true_card 3076706, qerror 2.2669125618961155, latency 0.01836085319519043
estimating query 2431: predicted 0.0, true_card 65723184, qerror 0.0, latency 0.08066606521606445
estimating query 2432: predicted 3463175.640984076, true_card 686665, qerror 5.043471912772715, latency 0.0705251693725586
estimating query 2433: predicted 0.0, true_card 61160701, qerror 0.0, latency 0.08767247200012207
estimating query 2434: predicted 3068886.334283391, true_card 655992, qerror 4.678237439303209, latency 0.07627177238464355
estimating query 2435: predicted 17144089.890318602, true_card 14929017, qerror 1.1483736598544032, latency 0.009079933166503906
estimating query 2436: predicted 98646.45490884902, true_card 38925, qerror 2.534269875628748, latency 0.009842634201049805
estimating query 2437: predicted 307948.3884312016, tr

estimating query 2495: predicted 26713.09573722691, true_card 77261, qerror 0.3457513588644583, latency 0.06295204162597656
estimating query 2496: predicted 7280568.0, true_card 4255848, qerror 1.7107208716100764, latency 0.005082845687866211
estimating query 2497: predicted 1161530.6829727187, true_card 948354, qerror 1.2247859796792324, latency 0.006426334381103516
estimating query 2498: predicted 0.0, true_card 2410386, qerror 0.0, latency 0.05958437919616699
estimating query 2499: predicted 33793.508211517226, true_card 185940, qerror 0.18174415516573747, latency 0.05866217613220215
estimating query 2500: predicted 0.0, true_card 3408749, qerror 0.0, latency 0.05833745002746582
estimating query 2501: predicted 9128.612192733865, true_card 110685, qerror 0.08247379674512233, latency 0.0587770938873291
estimating query 2502: predicted 686769.5592964238, true_card 210347, qerror 3.264936316165307, latency 0.015721797943115234
estimating query 2503: predicted 7019209.148816076, true_ca

estimating query 2552: predicted 8031.6584669411295, true_card 12124, qerror 0.6624594578473383, latency 0.057573795318603516
estimating query 2553: predicted 1521.1963461781268, true_card 60963, qerror 0.02495278031228986, latency 0.06283950805664062
estimating query 2554: predicted 11927.77462100279, true_card 172157, qerror 0.06928428481562057, latency 0.06334877014160156
estimating query 2555: predicted 12005177.262973124, true_card 438690, qerror 27.365969734831257, latency 0.016777992248535156
estimating query 2556: predicted 56575.08133517743, true_card 50824, qerror 1.113156802596754, latency 0.05872511863708496
estimating query 2557: predicted 9897.666037848237, true_card 249157, qerror 0.03972461555504456, latency 0.06191873550415039
estimating query 2558: predicted 187132.0054143246, true_card 659522, qerror 0.28373883724019006, latency 0.06392812728881836
estimating query 2559: predicted 8031.6584669411295, true_card 87188, qerror 0.09211885198583669, latency 0.064900636672

In [14]:
qerror = np.asarray(qerror)

In [15]:
temp_qerror = copy.deepcopy(qerror)
temp_qerror[temp_qerror < 1] = 1/temp_qerror[temp_qerror < 1]

/home/ubuntu/miniconda3/envs/bayescard/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in true_divide
  


In [16]:
for i in [50, 90, 95, 99, 100]:
    print(f"q-error {i}% percentile is {np.percentile(temp_qerror, i)}")

q-error 50% percentile is 3.9897887955326077
q-error 90% percentile is 41162.623558363804
q-error 95% percentile is nan
q-error 99% percentile is nan
q-error 100% percentile is nan


/home/ubuntu/miniconda3/envs/bayescard/lib/python3.7/site-packages/numpy/lib/function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


In [19]:
with open("stats_CEB_sub_queries_CE_scheme.txt", "w") as f:
    for p in pred:
        f.write(str(p)+"\n")

In [20]:
with open("stats_CEB_exec.sql", "r") as f:
    queries = f.readlines()

In [23]:
with open("stats_CEB_exec.sql", "w") as f:
    for q in queries:
        q = q.split("||")[-1]
        f.write(q)